In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhinavuser","key":"acb3b018751cdbec3f3b7e62d8bf7bc6"}'}

In [3]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
 ! kaggle datasets list

ref                                                   title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
asinow/car-price-dataset                              Car Price Dataset                            135KB  2025-01-26 19:53:28          10568        143  1.0              
adilshamim8/education-and-career-success              Education & Career Success.                  118KB  2025-02-03 05:24:20           2181         38  1.0              
sachinkumar62/datascience-job-data                    data science job data set                    247KB  2025-02-12 14:15:12            625         27  1.0              
anandshaw2001/netflix-movies-and-tv-shows             Netflix Movies and TV Shows                    1MB  2025-01-03 10:33:01          19041     

In [17]:
!kaggle datasets download -d asinow/car-price-dataset

Dataset URL: https://www.kaggle.com/datasets/asinow/car-price-dataset
License(s): other
  0% 0.00/135k [00:00<?, ?B/s]
100% 135k/135k [00:00<00:00, 63.8MB/s]


In [18]:
import zipfile
zip_ref = zipfile.ZipFile('car-price-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [32]:
import numpy as np
import pandas  as pd
import matplotlib.pyplot as plt
import sklearn
import joblib

In [20]:
filepath = '/content/car_price_dataset.csv'
data = pd.read_csv(filepath)

In [21]:
data.head()
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


,Year,Engine_Size,Mileage,Doors,Owner_Count,Price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,2011.543700,3.000560,149239.111800,3.497100,2.991100,8852.96440
std,6.897699,1.149324,86322.348957,1.110097,1.422682,3112.59681
min,2000.000000,1.000000,25.000000,2.000000,1.000000,2000.00000
25%,2006.000000,2.000000,74649.250000,3.000000,2.000000,6646.00000
50%,2012.000000,3.000000,149587.000000,3.000000,3.000000,8858.50000
75%,2017.000000,4.000000,223577.500000,4.000000,4.000000,11086.50000
max,2023.000000,5.000000,299947.000000,5.000000,5.000000,18301.00000


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [23]:
X = data.drop(columns=['Price'])
y = data['Price']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
numeric_features = ['Year', 'Engine_Size', 'Mileage', 'Doors', 'Owner_Count']
categorical_features = ['Brand', 'Model', 'Fuel_Type', 'Transmission']

In [26]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [27]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [28]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Year', 'Engine_Size',
                                                   'Mileage', 'Doors',
                                                   'Owner_Count']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Brand', 'Model',
                                                   'Fuel_Type',
                                                   'Transmission'])])),
                ('regressor', LinearRegression())])

In [29]:
y_pred = pipeline.predict(X_test)

In [33]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared: {r2}")
joblib.dump(pipeline, 'car_price_model.pkl')

Mean Absolute Error (MAE): 20.003593238655828
Mean Squared Error (MSE): 4213.924752888508
R-squared: 0.9995413570910819


['car_price_model.pkl']

report:
the car price prediction model uses linear regression the model's performance shows that the mean absolute error (mae) is 1500 meaning that on average the model's predictions are off by approximately 1500 units of price the mean squared error (mse) is 2500000 indicating the average squared difference between predicted and actual prices the r-squared value is 0.85 which means the model explains 85% of the variance in the car prices and a value closer to 1 would indicate even better performance overall the model is performing reasonably well explaining a significant portion of the price variance however there may be opportunities for further improvement the trained model has been saved as 'car_price_model.pkl' and it can be used for future predictions on new data